## Model description

This model corresponds to the "classical" bouncing ball. It is perhaps the simplest example of a hybrid ODE, since it presents only one discrete transition. The model consists of a ball dropped from a predefined height. It hits the ground after a certain time, loses energy and then bounces back into the air and starts to fall again.

This model has 2 continuous variables, 1 mode and 1 discrete jump.

The physical phenomena can be represented as the following hybrid automaton:

<img src="BouncingBall.png" alt="Drawing" style="width: 300px;"/>

In the discrete state, the motion of the ball, assumed to have a mass of $m = 1 kg$, is governed by the following differential equation:

$$
\begin{array} \\x'(t)= & v(t)\\ v'(t) = & -9.81 \end{array}
$$
where $x$ is the ball’s height from the ground, $v$ is the ball’s vertical velocity, and $9.81ms^{-2}$ is the earth’s gravitational force.

The invariant $x \geq 0$ enforces that the ball always bounces when it reaches the ground. The guard $x=0 \wedge v\leq0$ of the single discrete transition, which models the bouncing, ensures that bouncing happens after falling when reaching the ground. The corresponding reset condition $v:=-c \cdot v$ accounts for the loss of energy due to the ball’s deformation, where $c\in\left[0,1\right]$ is a constant known as [coefficient of restitution](https://en.wikipedia.org/wiki/Coefficient_of_restitution).

The model parameters and description are taken from the [HyPro collection of continuous and hybrid system benchmarks](https://ths.rwth-aachen.de/research/projects/hypro/).

In [1]:
using Revise # to debug
using Reachability, HybridSystems, MathematicalSystems, LazySets, LinearAlgebra

┌ Info: Recompiling stale cache file /Users/forets/.julia/compiled/v1.1/Reachability/CHV4V.ji for Reachability [774a0091-654f-5c65-bbdc-ad5b67b45832]
└ @ Base loading.jl:1184
ERROR: LoadError: LoadError: LoadError: UndefVarError: A_S not defined
Stacktrace:
 [1] top-level scope at /Users/forets/.julia/dev/Reachability/src/Utils/normalization.jl:120
 [2] include at ./boot.jl:326 [inlined]
 [3] include_relative(::Module, ::String) at ./loading.jl:1038
 [4] include at ./sysimg.jl:29 [inlined]
 [5] include(::String) at /Users/forets/.julia/dev/Reachability/src/Utils/Utils.jl:5
 [6] top-level scope at none:0
 [7] include at ./boot.jl:326 [inlined]
 [8] include_relative(::Module, ::String) at ./loading.jl:1038
 [9] include at ./sysimg.jl:29 [inlined]
 [10] include(::String) at /Users/forets/.julia/dev/Reachability/src/Reachability.jl:5
 [11] top-level scope at none:0
 [12] include at ./boot.jl:326 [inlined]
 [13] include_relative(::Module, ::String) at ./loading.jl:1038
 [14] include(::Modul

ErrorException: Failed to precompile Reachability [774a0091-654f-5c65-bbdc-ad5b67b45832] to /Users/forets/.julia/compiled/v1.1/Reachability/CHV4V.ji.

In [2]:
function bouncing_ball(;c = 0.75,
                       X0 = Interval(10.0, 10.2) × Interval(0, 0),
                       T = 40.0)

    # automaton structure
    automaton = LightAutomaton(1)

    # falling : x' = Ax + b, x ∈ X
    A = [0.0 1.0;
         0.0 0.0]
    b = [0.0,
        -9.81]
    X = HalfSpace([-1.0, 0.0], 0.0) # x >= 0
    falling = ConstrainedAffineContinuousSystem(A, b, X)
        
    # modes
    modes = [falling]

    # transition from mode falling to mode falling (self-loop)
    add_transition!(automaton, 1, 1, 1)
    
    # reset map (x, v) -> (x, -cv) with guard x == 0 and v <= 0
    A = [1.0 0.0;
         0.0 -c]
    
    guard = HPolyhedron([HalfSpace([0.0, 1.0], 0.0),   # v ≤ 0
                         HalfSpace([-1.0, 0.0], 0.0),  # x ≥ 0
                         HalfSpace([1.0, 0.0], 0.0)])  # x ≤ 0
    
    resetmaps = [ConstrainedLinearMap(A, guard)]

    # switching
    switchings = [AutonomousSwitching()]

    ℋ = HybridSystem(automaton, modes, resetmaps, switchings)

    # initial condition in "falling" mode
    initial_condition = [(1, X0)]

    problem = InitialValueProblem(ℋ, initial_condition)

    options = Options(:mode=>"reach", :T=>T, :plot_vars=>[1, 2])

    return (problem, options)
end

bouncing_ball (generic function with 1 method)

## Reachability settings

We consider the inital set given by an uncertain initial position and certain momentum (at rest):

$$
 x \in [10, 10.2], v = 0.
$$

We use a time horizon of $T=40s$ and set the constant $c = 0.75$.
The set of *bad states* is the set of all states where $v \geq 10.7$.

In [3]:
# settings
c = 0.75
X0 = Interval(10.0, 10.2) × Interval(0, 0)
T = 20.0

BouncingBall, options = bouncing_ball(c=c, X0=X0, T=T);
BadStates = HalfSpace([0.0, -1.0], -10.7); # v >= 10.7

UndefVarError: UndefVarError: Interval not defined

## Results

In [4]:
using Plots, LaTeXStrings

In [5]:
#=
doesn't work -- why?
@time begin
    sol = solve(BouncingBall, merge!(Options(:max_jumps=>0), options),
                BFFPSV18(:δ=>0.1, :block_options=>OctDirections));
end;
=#

@time begin
    sol = solve(BouncingBall, merge!(Options(:max_jumps=>0, :project_reachset=>false,
                                             :set_type_proj=>HPolygon, :verbosity=>1), options),
                BFFPSV18(:δ=>0.1, :partition=>[1:2], :block_options=>[HPolygon=>1e-2]));
end;

UndefVarError: UndefVarError: HPolygon not defined

In [6]:
sol.options.dict

UndefVarError: UndefVarError: sol not defined

In [7]:
sol.Xk[5]

UndefVarError: UndefVarError: sol not defined

In [8]:
plot(sol, xlab=L"x", ylab=L"v", alpha=.5)

UndefVarError: UndefVarError: sol not defined

In [9]:
plot(sol, xlab=L"x", ylab=L"v", alpha=.5)

UndefVarError: UndefVarError: sol not defined

In [10]:
# just checking the flowpipe for the "falling" mode:

In [11]:
# falling : x' = Ax + b, x ∈ X
A = [0.0 1.0;
     0.0 0.0]
b = [0.0,
    -9.81]
U = Singleton([1.0])

problem = IVP(AffineContinuousSystem(A, b), X0)
options = Options(:mode=>"reach", :T=>10, :plot_vars=>[1, 2])

solve(problem, options, op=BFFPSV18(:δ=>1.0));

UndefVarError: UndefVarError: Singleton not defined

In [12]:
AffineContinuousSystem(A, b)

UndefVarError: UndefVarError: AffineContinuousSystem not defined